<a href="https://colab.research.google.com/github/FarshidNooshi/Information-Retrieval/blob/master/notebooks/phase2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%pip install parsivar 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 36.2 MB 250 kB/s 
     |████████████████████████████████| 1.5 MB 37.9 MB/s 
  Created wheel for parsivar: filename=parsivar-0.2.3-py3-none-any.whl size=36492972 sha256=6b9a54f7297b0356eac7cc5c99803ff2035fff50bcfaffd7d77f2e3ee2c41013
  Stored in directory: /root/.cache/pip/wheels/ae/67/7a/49cbf08f64d3f76a26eceaf0e481a40e233f05d4356875cbed
  Created wheel for nltk: filename=nltk-3.4.5-py3-none-any.whl size=1449923 sha256=fdbc36cbec980d06e4cd9549efb80426b03c0cd97155ad3ebe99322876f280f5
  Stored in directory: /root/.cache/pip/wheels/48/8b/7f/473521e0c731c6566d631b281f323842bbda9bd819eb9a3ead
Successfully built parsivar nltk
  Attempting uninstall: nltk
    Found existing installation: nltk 3.2.5
    Uninstalling nltk-3.2.5:
      Successfully uninstalled nltk-3.2.5


In [2]:
%pip install matplotlib
from matplotlib import pyplot as plt

# coding: utf8
import codecs
from os import path


class Config:
    def __init__(self):
        self.config = {
            'champions_list': False,
            'champions_lists_ratio': 0.5,
            'documents_to_show': 10,
            'documents': [],
            'show_heaps_law': False,
            'show_zipf_law': False,
            'documents_path': 'Phase_1/assets/IR_data_news_12k.json',
            'dump_path': 'Phase_2/assets/',
            'remove_stop_words': True,
            'do_stemming': True,
        }

    def get_config(self, config_name):
        return self.config[config_name]

    def set_config(self, config_name, value):
        self.config[config_name] = value


default_stop_words = path.join('/content', 'stopwords.dat')


class StopWord:
    """ Class for remove stop words

         """

    def __init__(self, file_path=default_stop_words, normal=False):
        self.file_path = file_path
        self.normal = normal
        self.normalizer = Normalizer().normalize
        self.stop_words = self.init(file_path, normal)

    def init(self, file_path, normal):
        if not normal:
            return set(
                line.strip("\r\n") for line in codecs.open(file_path, "r", encoding="utf-8").readlines())
        else:
            return set(
                self.normalizer(line.strip("\r\n")) for line in
                codecs.open(file_path, "r", encoding="utf-8").readlines())

    def set_normalizer(self, func):
        self.normalizer = func
        self.stop_words = self.init(self.file_path, self.normal)

    def __getitem__(self, item):
        return item in self.stop_words

    def __str__(self):
        return str(self.stop_words)

    def clean(self, iterable_of_strings, return_generator=False):
        if return_generator:
            return filter(lambda item: not self[item], iterable_of_strings)
        else:
            return list(filter(lambda item: not self[item], iterable_of_strings))


class Document:
    def __init__(self, content, url, title):
        self.content = content
        self.url = url
        self.title = title

# coding: utf8
from __future__ import unicode_literals

import json
import pickle


from parsivar import Normalizer, Tokenizer, FindStems

def read_file(path='/content/IR_data_news_12k.json'):
    documents_title = []
    documents_content = []
    documents_url = []
    with open(path, encoding='UTF-8') as f:
        data = json.load(f)
        for i in data:
            documents_title.append(data[i]["title"])
            documents_content.append(data[i]["content"])
            documents_url.append(data[i]['url'])
    return documents_url, documents_title, documents_content


normalizer = Normalizer()


def preprocess_pipeline(content, configurations):
    str_empty = ' '
    stemmer = FindStems()
    tokenizer = Tokenizer()
    normalized = normalizer.normalize(content)
    result = tokenizer.tokenize_words(normalized)
    if configurations.get_config('do_stemming'):
        result = list(map(stemmer.convert_to_stem, result))
    if configurations.get_config('remove_stop_words'):
        result = StopWord(normal=False).clean(content)
    result = str_empty.join(content)
    return content


def heaps_law(dict_size, num_total_tokens, number):
    print(f'at {number}th document we seen {num_total_tokens} total tokens and dictionary size was {dict_size}')


def print_results(results):
    for single_result in results:
        print(f"score: {single_result['score']},\n"
              f"url: {single_result['url']},\n"
              f"title: {single_result['title']}\n"
              f'content: {single_result["content"]}\n*********************************************************\n\n')


def zipf_law(frequencies, title):
    from matplotlib import pyplot as plt

    frequencies.sort(reverse=True)

    # enumerate the ranks and frequencies
    rf = [(r + 1, f) for r, f in enumerate(frequencies)]
    rs, fs = zip(*rf)

    plt.clf()
    plt.xscale('log')
    plt.yscale('log')
    plt.title(title)
    plt.xlabel('rank')
    plt.ylabel('frequency')
    plt.plot(rs, fs, 'b-')
    plt.show()


def save_index(configurations, pos_index):
    with open(f'../{configurations.get_config("dump_path")}/updated_pos_index.pkl', 'w') as file_to_write:
        pickle.dump(pos_index, file_to_write, pickle.HIGHEST_PROTOCOL)
        print(f'saved index with name updated_pos_index.pkl')


def load_index(configurations):
    index = None
    with open(f'../{configurations.get_config("dump_path")}/updated_pos_index.pkl', 'rb') as file_to_read:
        index = pickle.load(file_to_read)
    return index

# save_index(config)
# pos_index = load_index(config)



class SimplePositionalIndex:
    def __init__(self, config):
        self.config = config
        self.Documents = config.get_config('documents')
        self.positional_index_structure = {}
        self.url_to_information = {}
        self.show = config.get_config('show_heaps_law')
        self.build_positional_index()
        self.build_url_to_information_dict()

    def build_positional_index(self):
        num_tokens = 0
        x = []
        y = []
        for i, document in enumerate(self.Documents):
            preprocessed_content = preprocess_pipeline(document.content, self.config)
            processed_content = process_positions(preprocessed_content)
            title_of_document = document.title
            url = document.url
            num_tokens += len(processed_content.keys())
            for word, positions in processed_content.items():
                self.add_index(word, positions, title_of_document, url)
            if i % 500 or not i or i > 2000:
                continue
            x.append(len(self.positional_index_structure.keys()))
            y.append(num_tokens)
            if self.show:
                heaps_law(dict_size=len(self.positional_index_structure.keys()), num_total_tokens=num_tokens, number=i)
        if self.show:
            heaps_law(dict_size=len(self.positional_index_structure.keys()), num_total_tokens=num_tokens,
                      number=len(self.Documents))
            plt.clf()
            plt.title('With Stemming')
            plt.plot(x, y, 'g-')
            plt.show()

    def add_index(self, word, positions, title, url):
        index_to_add = generate_index(positions, title)
        if word not in self.positional_index_structure:
            self.positional_index_structure[word] = {"total occurrences": 0,
                                                     "indexes": {}}
        self.positional_index_structure[word]["total occurrences"] += index_to_add[
            "number of occurrences in document"]
        self.positional_index_structure[word]["indexes"][url] = index_to_add

    def build_url_to_information_dict(self):
        for document in self.Documents:
            self.url_to_information[document.url] = {"title": document.title,
                                                     "content": document.content}


def process_positions(preprocessed_content):
    positions = {}
    for position_of_word, word in enumerate(preprocessed_content.split()):
        if word and word not in positions:
            positions[word] = []
        positions[word].append(position_of_word)
    return positions


def generate_index(positions, title):
    return {"number of occurrences in document": len(positions),
            "positions": positions,
            "title of document": title}


def docID(number, lst_urls):
    return lst_urls[number]


def position(plist):
    return plist['positions']


def pos_intersect(p1, p2, k):
    answer = {}  # answer <- ()
    len1 = len(p1)
    len2 = len(p2)
    i = j = 0
    lst_urls1 = sorted(list(p1.keys()))
    lst_urls2 = sorted(list(p2.keys()))
    while i != len1 and j != len2:
        key = docID(i, lst_urls1)
        key2 = docID(j, lst_urls2)
        if key == key2:
            l = []
            pp1 = position(p1[key])
            pp2 = position(p2[key])

            plen1 = len(pp1)
            plen2 = len(pp2)
            ii = jj = 0
            while ii != plen1:
                while jj != plen2:
                    if abs(pp1[ii] - pp2[jj]) <= k:
                        l.append(pp2[jj])
                    elif pp2[jj] > pp1[ii]:
                        break
                    jj += 1
                # l.sort()
                while l != [] and abs(l[0] - pp1[ii]) > k:
                    l.remove(l[0])
                for ps in l:
                    if key in answer:
                        # answer[key]['positions'].extend([pp1[ii], ps])
                        answer[key]['positions'].extend([max(pp1[ii], ps)])
                    else:
                        answer[key] = {'positions': [max(pp1[ii], ps)]}
                ii += 1
            i += 1
            j += 1
            if key in answer:
                answer[key]['positions'] = list(set(answer[key]['positions']))
        elif key < key2:
            i += 1
        else:
            j += 1
    return answer


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
import math

class DevelopedPositionalIndex(SimplePositionalIndex):
    def __init__(self, configurations):
        super().__init__(configurations)
        self.total_number_of_documents = len(self.Documents)
        self.document_term_tfidf_dictionary = {}
        self.build_updated_positional_index()
        if configurations.get_config('champions_list'):
            self.champions_list = self.build_champions_list()

    def build_updated_positional_index(self):
        for WORD in self.positional_index_structure.keys():
            for DOC_URL, DICTIONARY in self.positional_index_structure[WORD]['indexes'].items():
                DICTIONARY['tf idf'] = self.get_tf_value(WORD, DOC_URL) * self.get_idf_value(WORD)
                if DOC_URL not in self.document_term_tfidf_dictionary.keys():
                    self.document_term_tfidf_dictionary[DOC_URL] = {WORD: DICTIONARY['tf idf']}
                else:
                    self.document_term_tfidf_dictionary[DOC_URL][WORD] = DICTIONARY['tf idf']

    def get_tf_value(self, word, url):
        return math.log(1 + self.positional_index_structure[word]['indexes'][url]['number of occurrences in document'])

    def get_idf_value(self, word):
        return math.log(self.total_number_of_documents / len(self.positional_index_structure[word]['indexes']))

    def build_champions_list(self):
        champions_list = {}
        for WORD in self.positional_index_structure.keys():
            url_tf_dictionary = {}
            for DOC_URL, DICTIONARY in self.positional_index_structure[WORD]['indexes'].items():
                url_tf_dictionary[DOC_URL] = self.get_tf_value(WORD, DOC_URL)
            champions_list_size = int(
                self.config.get_config('champions_lists_ratio') * len(self.positional_index_structure[WORD]['indexes']))
            champions_list[WORD] = sorted(url_tf_dictionary, key=lambda item: item[1], reverse=True)[
                                   :champions_list_size]
        return champions_list



In [ ]:
config = Config()
config.set_config('documents_path',
                  '/content/IR_data_news_12k.json')
docs_url, docs_title, docs_content = read_file(config.get_config('documents_path'))

config.set_config('documents',
                  [Document(content, url, title) for url, title, content in zip(docs_url, docs_title, docs_content)])
config.set_config('champions_list', True)

In [ ]:
pos_index = DevelopedPositionalIndex(config)

In [ ]:
from collections import Counter
from math import log


class QueryHandler:
    def __init__(self, positional_index, config):
        self.positional_index = positional_index
        self.config = config

    def answer_query(self, query):
        query = preprocess_pipeline(query, self.config)
        print(f'Query: {query}\n\n\n')
        terms = query.split()
        vector_values = self.tf_idf_calculate(terms, self.config.get_config('champions_list'))
        scores = self.calculate_scores(vector_values)
        results_url = dict(sorted(scores.items(), key=lambda item: item[1], reverse=True)[
                           :self.config.get_config('documents_to_show')])
        total_results = self.generate_total_results(results_url)
        return total_results

    def generate_total_results(self, results_url):
        total_results = []
        for URL, SCORE in results_url.items():
            result = self.positional_index.url_to_information[URL]
            result['score'] = SCORE
            result['url'] = URL
            total_results.append(result)
        return total_results

    def tf_idf_calculate(self, terms, champions_list):
        vector_values = {}
        tf_values = Counter(terms)
        for term in terms:
            for DOC_URL in self.get_search_dict_list(champions_list, term):
                if DOC_URL not in vector_values.keys():
                    vector_values[DOC_URL] = {}
                vector_values[DOC_URL][term] = \
                    (log(1 + tf_values[term])) * self.positional_index.get_idf_value(term)
        return vector_values

    def get_search_dict_list(self, champions_list, term):
        if champions_list and term in self.positional_index.champions_list.keys():
            return self.positional_index.champions_list[term]
        if term in self.positional_index.positional_index_structure.keys():
            return self.positional_index.positional_index_structure[term]['indexes']
        return None

    @staticmethod
    def cosine_similarity(v1, v2):
        dot_product = 0
        for term in v1.keys():
            if term in v2.keys():
                dot_product += v1[term] * v2[term]
        magnitude_v1 = 0
        for term in v1.keys():
            magnitude_v1 += v1[term] ** 2
        magnitude_v2 = 0
        for term in v2.keys():
            magnitude_v2 += v2[term] ** 2
        return dot_product / (magnitude_v1 ** 0.5) / (magnitude_v2 ** 0.5)

    def calculate_scores(self, vector_values):
        scores = {}
        for DOC_URL, TERM_SCORES in vector_values.items():
            scores[DOC_URL] = self.cosine_similarity(vector_values[DOC_URL],
                                                     self.positional_index.document_term_tfidf_dictionary[DOC_URL])
        return scores



In [ ]:
query_handler = QueryHandler(pos_index, config)


In [ ]:
print_results(query_handler.answer_query('مجلس'))

Query: مجلس



score: 0.14876352619368935,
url: https://www.farsnews.ir/news/14001022000190/دشمن-حدود-0-هزار-نفر-را-برای-تخریب-مجلس-به-کار-گرفته-است,
title: دشمن حدود 170 هزار نفر را برای تخریب مجلس به کار گرفته است
content: 
حجت الاسلام علیرضا سلیمی نماینده مردم محلات در مجلس شورای اسلامی در گفت‌وگو با خبرنگار پارلمانی خبرگزاری فارس، با اشاره به نشست غیرعلنی امروز مجلس گفت: در جلسه امروز اقدامات دشمن برای تخریب مجلس یازدهم مورد بحث قرار گرفت. وی افزود: واقعیت این است که دشمن بین 170 تا 180 هزار نفر نیرو را برای تخریب مجلس یازدهم و ایجاد انحراف در افکار عمومی و ایجاد یأس در میان مردم نسبت به عملکرد مجلس یازدهم به کار گرفته است. عضو هیات رئیسه مجلس اظهار داشت: این تخریب از رئیس مجلس شروع شده و به کلیت مجلس و تک تک نمایندگان نیز ختم می‌شود که عمده افرادی که در این حوزه فعال هستند در خارج از کشور و بخشی هم در داخل کشور هستند. انتهای پیام/



*********************************************************


score: 0.1396390266176801,
url: https://www.farsnews.ir/news/14001118000150/بررسی-لایحه-ب

In [ ]:
print_results(query_handler.answer_query('قهرمانی تیم ملی ایران'))

Query: قهرمانی تیم ملی ایران



score: 0.21998287334939895,
url: https://www.farsnews.ir/news/14001111000880/قطر-با-غلبه-بر-بحرین-بر-بام-هندبال-آسیا-ایستاد,
title: قطر با غلبه بر بحرین بر بام هندبال آسیا ایستاد
content: 
به گزارش خبرگزاری فارس، در فینال رقابت های هندبال قهرمانی آسیا، قطر و بحرین به مصاف هم رفتند که قطری ها موفق شدند با نتیجه 29 بر 24 به پیروزی برسند و عنوان قهرمانی را بدست آوردند. این پنجمین قهرمانی قطر در رقابت های قهرمانی آسیا به شمار می رود.    بحرین در جایگاه دوم ایستاد و عربستان هم که در دیدار رده بندی مقابل هندبال کشورمان پیروز شده بود جایگاه سوم را بدست آورد. تیم ملی کشورمان در رده چهارم مسابقات قهرمانی آسیا ایستاد و کره جنوبی هم پنجم شد. انتهای پیام/



*********************************************************


score: 0.2102203031565961,
url: https://www.farsnews.ir/news/14001110000953/هندبال-قهرمانی-اروپا|-سوئد-در-دیداری-جذاب-مقابل-اسپانیا-قهرمان-شد,
title: هندبال قهرمانی اروپا| سوئد در دیداری جذاب مقابل اسپانیا قهرمان شد
content: 
به گزارش خبرگزاری فارس، دید

In [ ]:
print_results(query_handler.answer_query('اضطراب'))

Query: اضطراب



score: 0.16707285722382625,
url: https://www.farsnews.ir/news/14001129000331/مدیرعامل-نساجی-برای-استان-مهم-نیست-زمین-داریم-یا-نه-قلعه‌نویی-به,
title: مدیرعامل نساجی: برای استان مهم نیست زمین داریم یا نه/قلعه‌نویی به نساجی عرق دارد
content: 
به گزارش خبرگزاری فارس از مشهد، ایزد سیف‌الله‌پور درباره استقبال از امیر قلعه‌نویی، عنوان کرد: قلعه‌نویی افتخار فوتبال ایران است و من وظیفه دارم به عنوان میزبان به او خوشامد بگویم. ما دوست هستیم و رابطه خوبی با هم داریم. قلعه‌نویی برای بازی ما در مشهد زحمت زیادی کشید. او از سالها قبل پیگیر فوتبال نساجی بوده و از حامیان ما. او به نساجی عرق دارد. مدیرعامل نساجی افزود: صحبت قلعه‌نویی این بود به جز بازی با پرسپولیس و استقلال اینجا اگر تماشاگر داشته باشد به مشکل برمی خوریم. باید از تربیت بدنی خراسان رضوی، استاندار، شورای تامین، هیات فوتبال و ... تشکر کنم که کمک کردند بازی‌ها اینجا برگزار شود. ما در بازی قبلی اضطراب داشتیم و تیم در زمین بود اما نمی‌دانستیم بازی برگزار می‌شود یا خیر. به خاطر نبود یگان ویژه ترس داشتند و همین کار را برای بچه

In [ ]:
print_results(query_handler.answer_query('تیبو کورتوا'))

Query: تیبو کورتوا



score: 0.5174940362895477,
url: https://www.farsnews.ir/news/14001213000563/ستاره-بلژیکی-بهترین-بازیکن-ماه-رئال-مادرید,
title: ستاره بلژیکی بهترین بازیکن ماه رئال مادرید
content: 
به گزارش خبرگزاری فارس، تیبو کورتوا سنگربان بلژیکی باشگاه رئال مادرید به عنوان بهترین ماه فوریه باشگاه رئال مادرید انتخاب شد. وی همچنین به عنوان یکی از نامزدهای کسب عنوان بهترین بازیکن ماه لالیگا هم انتخاب شده است. انتهای پیام/



*********************************************************


score: 0.16275315588139153,
url: https://www.farsnews.ir/news/14001218000985/ترکیب-منتخب-رئال-و-پاریس-به-بهانه-دیدار-حساس-امشب-در-لیگ-قهرمانان-عکس,
title: ترکیب منتخب رئال و پاریس به بهانه دیدار حساس امشب در لیگ قهرمانان+عکس
content: 
به گزارش خبرگزاری فارس، امشب از ساعت 23:30 به وقت ایران، رئال مادرید در ورزشگاه سانتیاگو برنابئو میزبان پاری سن ژرمن است و این بازی ساعت 23:30 به وقت ایران آغاز می شود. دیدار رفت با گل دیرهنگام کیلیان ام باپه مهاجم پاری سن ژرمن با یک گل به سود این تیم به پایان رسیده است و